In [1]:
%pip install langchain openai chromadb --quiet

In [9]:
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA

In [2]:
loader = CSVLoader(file_path="co2_mm_gl.csv")
data = loader.load()

In [5]:
csv_index = VectorstoreIndexCreator().from_loaders([loader])

Using embedded DuckDB without persistence: data will be transient


In [10]:
chain = RetrievalQA.from_chain_type(
    llm=OpenAI(),
    chain_type="stuff",
    retriever=csv_index.vectorstore.as_retriever(),
    input_key="question",
)

In [18]:
chain({"question": "When was the minimum uncertainty?"})

{'question': 'When was the minimum uncertainty?',
 'result': ' The minimum uncertainty was in June 2002, with an average uncertainty of 0.05 and a trend uncertainty of 0.04.'}

In [19]:
chain({"question": "What was the uncertainty in May 1979?"})

{'question': 'What was the uncertainty in May 1979?',
 'result': ' The uncertainty in May 1979 was 0.04.'}